# Session 2

## Contents: 
- First-Class Citizens in programming
  - Higher-Order Functions
  - Closures & Free Variables
  - Pipeline Showcase
- Decorators
- Lambda expressions
- Comprehensions


## First-Class Functions
A programming language is said to have first-class functions if treats functions as _"First-class Citizens"_

### But what is a __*"First-Class Citizen"*__ in programming?
A first-class citizen in a programming language is an entity which supports all the operations generally available to other entities. These operations typically include being passed as an argument, returned from a function, and assigned to a variable.

In [ ]:
# Assigning function to a variable

# A function that returns the square of a given number
def square(x):
    return x*x


f = square
print(square)
print(f(5))

## Higher order functions
A function that takes a function as an argument or returns a function as a result is called a _"higher order function"_.

In [ ]:
# Let's build our own map function!
def my_map(func, iterable):
    result = []
    for i in iterable:
        result.append(func(i))
    return result

print(my_map(square, [1, 2, 3, 4, 5]))

# # Exercise:
# # Todo 1: Move to comprehension exercises
# # Apply list comprehesions to the map function that we just wrote.

# # def my_map(func, iterable):
# #     return [func(i) for i in iterable]

# # Let's try it out!
# print(my_map(square, [1, 2, 3, 4, 5]))

def cube(num):
    return num ** 3

print(my_map(cube, [1, 2, 3, 4, 5]))



In [ ]:
# Returning a function from another function
def logger(msg):
    def log_message():
        print('Log:', msg)
    return log_message


# Let's say Hi!
log_hi = logger('Hi!')
# print(log_hi())
print(log_hi.__name__)
log_hi()

In [ ]:
# HTML tag generator
def html_tag(tag):
    def wrap_text(msg):
        return f'<{tag}>{msg}</{tag}>'
    return wrap_text

h1 = html_tag('h1')
print(h1.__name__)
print(h1('Test Headline'))

print(h1('Another Headline!'))

p = html_tag('p')
print(p('Test Paragraph'))


In [ ]:
# Exercise:
# Todo 2: Write a function that takes a function as an argument and returns a function.
def operate(func, *args):
    return func(*args)
def add(x, y):
    return x + y
my_function = operate(square, 5)
print(my_function)
print(operate(square, 5))

## Closures
A closure is a function that remembers values in enclosing scopes even if they are _not present_ in memory when the function is called.

## Free variables
Variables that can still be refrenced in the body of a function even if it is not present in memory when the function is called.

In [ ]:
# An example of closure
def outter_func():
    # message is a free variable!
    message = 'Hi!'
    def inner_func():
        print(message)
    return inner_func

func = outter_func()
print(func.__name__)
func()

In [ ]:
def another_outter_func(msg):
    message = msg
    def inner_func():
        print(message)
    return inner_func
hi = another_outter_func('Hi!')
bye = another_outter_func('Bye!')
hi()
bye()

### Note:
A closure closes over the free variables in its enclosing scope.

In [ ]:
# Pipeline Showcase!

def pipeline_component(func, data, args):
    return func(data, *args)


def pipeline(data: list, components: list):

    # Python treats list, dict, tuple, and set objects as passed reference, thus
    # in order to not modify the previous state, we make a local copy
    if (
        isinstance(data, list)
        or
        isinstance(data, tuple)
        or 
        isinstance(data, set)
        or 
        isinstance(data, dict)
    ):
        __data = data.copy()
    else:
        __data = data
    # A mapping of different filters possible
    function_mappings = {
        "square": square,
        "cube": cube,
        "add": add,
        # Simply add the mapping of a new function,
        # nothing else will really need to changed
    }

    # Going through each of the components
    for component in components:

        # If component is simply empty, continue to next
        # iteration
        if component == {}:
            continue

        # Send to pipeline component, return data to `__data`
        __data = pipeline_component(
            # Map function respectively using the function_mappings dictionary
            func=function_mappings[f'{component["filter"]}'],
            # Send over the data
            data=__data,
            # Except the filter name, select the rest as args
            args=tuple(list(component.values())[1:]),
        )

    # Return the data
    return __data

def add(x, y):
    return x + y

data = pipeline(
    data=2,
    components=[
        {"filter": "square"},
        {"filter": "cube"},
        {"filter": "add", "num": 3},
    ]
)
print(data)


## Decorators
Decorators are a way to modify the behavior of a function without modifying the function itself. It takes a function as a parameter and returns a modified function with the altered/added behavior.

In [ ]:
def decorator_function(original_function):
    def wrapper_function():
        print('wrapper executed this before "{}"'.format(original_function.__name__))
        return original_function()
    return wrapper_function

def display():
    print('display function ran')

decorated_display = decorator_function(display)
decorated_display()

# ! This is kinda a wacky syntax, decorators can be way cooler than this!

In [ ]:
# ! Behold, the cooler way to do this!
@decorator_function
def display():
    print('display function ran')

display()

In [ ]:
# What if we want to decorate a function that already has arguments?
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Omar', 22)

# Let's try decorating it with the decorator_function decortor
@decorator_function
def display_info_with_arguments(name, age):
    print('display_info_with_arguments ran with arguments ({}, {})'.format(name, age))

display_info_with_arguments('Omar', 22)

# Will this work? why/why not? No, it won't work because wrapper function does not
# accept any args!


In [ ]:
def decorator_function_with_parameters(original_function):
    def wrapper_function(*args, **kwargs):
        print('wrapper executed this before "{}"'.format(original_function.__name__))
        return original_function(*args, **kwargs)
    return wrapper_function

# Let's try decorating it with the new decortor now
@decorator_function_with_parameters
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Omar', 22)

## Decorators as Classes

In [ ]:
# Decorator class
class decorator_class(object):
    def __init__(self, original_function):
        self.original_function = original_function
    
    def __call__(self, *args, **kwargs):
        print('call method executed this before "{}"'.format(self.original_function.__name__))
        return self.original_function(*args, **kwargs)
    
# Let's try decorating it with the new decortor class now
@decorator_class
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Omar', 22)


### Practical Examples for Using Decorators

In [ ]:
# Keeping track of different functions activities using logging
from functools import wraps
def logger(func):
    import logging
    logging.basicConfig(filename=f'{func.__name__}.log', level=logging.INFO)

    @wraps(func)
    def wrapper(*args, **kwargs):
        logging.info(f'Ran with args: {args}, and kwargs: {kwargs}')
        return func(*args, **kwargs)
    return wrapper

@logger
def display_info(name, age):
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Aya', 18)
display_info('Mariam', 21)


In [ ]:
# Track the time it takes to run a function

from time import time, sleep

def time_it(func):
    print(func.__name__)

    @wraps(func)
    def wrapper(*args, **kwargs):
        t1 = time()
        result = func(*args, **kwargs)
        t2 = time()
        print(f'{func.__name__} ran in {t2-t1} seconds')
        return result
    return wrapper

@time_it
def display_info(name, age):
    sleep(1)
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Mazen', 22)


In [ ]:
# Using multiple decorators
from time import sleep

@time_it
@logger
def display_info(name, age):
    sleep(1)
    print('display_info ran with arguments ({}, {})'.format(name, age))

display_info('Michael', 20)

## Lambda expressions
Lambda expressions are a way to create anonymous functions in Python.

### Anonymous Functions
Functions that are defined without a name are called _"anonymous functions"_ or _"lambda functions"_.

In [ ]:
# Difference between regular functions and lambda functions
# Regular functions
def add(x, y):
    return x + y
print(add(3, 4))

# Lambda functions
lambda_add = lambda x, y: x + y
print(lambda_add(3, 4))

# How can nameless functions be useful?
# ! This is a very common use case
squares = my_map(lambda x: x*2, [1, 2, 3, 4, 5])
print(squares)

# Combine first and last name into a single "Full Name"
full_name = lambda fn, ln: fn.strip().title() + " " + ln.strip().title()
print(full_name("   Ahmed   ", "   Ali   "))

In [ ]:
# Lambda functions as a key to sort a list of dictionaries
# ! This is a very common use case
people = [
    {"name": "Michael", "age": 20},
    {"name": "Marwa", "age": 19},
    {"name": "Ahmed", "age": 25},
    {"name": "Sohila", "age": 23}
]

# sorting people by last name
people.sort(key=lambda person: person['age'])
print(people)

scifi_authors = [
    "Isaac Asimov", 
    "Ray Bradbury", 
    "Robert Heinlein", 
    "Arthus C. Clarke", 
    "Frank Herbert", 
    "Orson Scott Card", 
    "Douglas Adams", 
    "H. G. Wells", 
    "Leigh Brackett"
]

# Case insensitive sorting by last name
scifi_authors.sort(key=lambda name: name.split(" ")[-1].lower())
print(scifi_authors)

## Note:
Lambda expressions cannot be used as multi-line functions.